# CRUD Operations and Transaction Management

With a connection established, we can now perform the fundamental database operations, collectively known as **CRUD**: Create, Read, Update, and Delete.

This notebook will also cover the most critical concept in database reliability: **transactions**. We will learn how to use `connection.commit()` to save changes and `connection.rollback()` to discard them, ensuring that our database remains in a consistent state.

--- 
## Setup

We'll import `psycopg2` and define our connection details, just as before.

In [1]:
import psycopg2

DB_HOST = "localhost"
DB_NAME = "people"
DB_USER = "fahad"
DB_PASS = "secret"

--- 
## Step 1: Create a Sample Table

Let's create a simple `employees` table to use for our CRUD examples. We'll use the `with` statement for clean execution.

In [2]:
sql = """
DROP TABLE IF EXISTS employees CASCADE;
CREATE TABLE employees (
    id SERIAL PRIMARY KEY,
    name VARCHAR(255) NOT NULL,
    role VARCHAR(255),
    salary INT
);
"""

try:
    with psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS) as conn:
        with conn.cursor() as cur:
            cur.execute(sql)
    print("Table 'employees' created successfully.")
except psycopg2.Error as e:
    print(f"Database error: {e}")

Table 'employees' created successfully.


--- 
## Step 2: CRUD Operations

Now we'll perform each of the CRUD operations. Notice that after every data-modifying operation (`INSERT`, `UPDATE`, `DELETE`), we must call `conn.commit()` to make the changes permanent.

In [3]:
try:
    with psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS) as conn:
        with conn.cursor() as cur:
            # CREATE: Insert a new employee
            print("--- Inserting a new employee ---")
            cur.execute("INSERT INTO employees (name, role, salary) VALUES (%s, %s, %s);", ('Alice', 'Developer', 80000))
            # We'll commit at the end of the block.

            # READ: Fetch and display the employee
            print("--- Reading the data ---")
            cur.execute("SELECT * FROM employees WHERE name = %s;", ('Alice',))
            print(cur.fetchone())
            
            # UPDATE: Give Alice a raise
            print("\n--- Updating Alice's salary ---")
            cur.execute("UPDATE employees SET salary = %s WHERE name = %s;", (90000, 'Alice'))
            cur.execute("SELECT * FROM employees WHERE name = %s;", ('Alice',))
            print(cur.fetchone())

            # DELETE: Remove Alice from the table
            print("\n--- Deleting Alice ---")
            cur.execute("DELETE FROM employees WHERE name = %s;", ('Alice',))
            cur.execute("SELECT COUNT(*) FROM employees;")
            print(f"Total employees now: {cur.fetchone()[0]}")

        # The transaction is committed here, when the 'with conn' block exits successfully
        # Or you can explicitly call conn.commit()
        conn.commit()
        print("\nChanges committed successfully.")
except psycopg2.Error as e:
    print(f"Database error: {e}")

--- Inserting a new employee ---
--- Reading the data ---
(1, 'Alice', 'Developer', 80000)

--- Updating Alice's salary ---
(1, 'Alice', 'Developer', 90000)

--- Deleting Alice ---
Total employees now: 0

Changes committed successfully.


--- 
## Step 3: Understanding `commit()` and `rollback()`

A **transaction** is a sequence of operations performed as a single logical unit of work. All changes in a transaction are temporary until you **commit** them. You can discard all changes in a transaction by performing a **rollback**.

Let's demonstrate this by trying to add an employee but then rolling back the change.

In [4]:
try:
    with psycopg2.connect(host=DB_HOST, dbname=DB_NAME, user=DB_USER, password=DB_PASS) as conn:
        with conn.cursor() as cur:
            # Check initial state
            cur.execute("SELECT COUNT(*) FROM employees WHERE name = 'Bob';")
            print(f"Employees named Bob before insert: {cur.fetchone()[0]}")
            
            # Start a transaction and insert a new employee
            cur.execute("INSERT INTO employees (name, role, salary) VALUES (%s, %s, %s);", ('Bob', 'Manager', 120000))
            
            # We can see Bob within our own transaction
            cur.execute("SELECT COUNT(*) FROM employees WHERE name = 'Bob';")
            print(f"Employees named Bob during transaction: {cur.fetchone()[0]}")

            # Now, instead of committing, we roll back
            print("--- Rolling back the transaction ---")
            conn.rollback()
            
            # Check the final state
            cur.execute("SELECT COUNT(*) FROM employees WHERE name = 'Bob';")
            print(f"Employees named Bob after rollback: {cur.fetchone()[0]}")
            
except psycopg2.Error as e:
    print(f"Database error: {e}")

Employees named Bob before insert: 0
Employees named Bob during transaction: 1
--- Rolling back the transaction ---
Employees named Bob after rollback: 0


--- 
## Conclusion

In this notebook, we learned how to perform the four fundamental CRUD operations. More importantly, we learned about the atomic nature of database transactions:

- By default, `psycopg2` starts a new transaction when you execute your first command.
- Changes are only made permanent and visible to other connections when you call `connection.commit()`.
- You can discard all changes made since the last commit by calling `connection.rollback()`.
- Using a `with psycopg2.connect(...) as conn:` block will automatically `commit` on successful exit or `rollback` if an exception occurs.

This transactional control is essential for writing reliable and robust database applications.